### Fitness4All Business Location Prospecting Project

##### Introduction/Business Problem 

Fitness4All operates gyms in the state of New Jersey. The company is looking to expand into Pennsylvania and would like to identify a shortlist of locations (townships) with the best prospects for profitability.  

##### Data 

Fitness4All believes that the most profitable locations for expansion are townships currently under-served by commercial gym facilities with a population of between 20,000 and 50,000. This project will use readily available population data from Wikipedia to identify a baseline list of possible locations (https://en.wikipedia.org/wiki/List_of_populated_places_in_Pennsylvania). 

It will then use Foursquare location data to explore existing locations for gym facilities in these townships, to discover the competitive landscape. We will start by retrieving information on venues for all townships on the baseline list so that we can calculate the density of gyms relative to population size. This should enable us to winnow down the initial list of about 20 towns to a smaller focus list of potential locations. For the focus list, we can then evaluate potential cross-street sites that are i. somewhat remote from competitive facilities; ii. close to complementary amenities e.g. shops, restaurants that support the integration of gym sessions with other frequent daily activities. Below is a plot of gym facilities in Altoona PA to illustrate the Foursquare data we will be using. Altoona may or may not be a good candidate for the focus list of prospective locations - the full analysis will determine this.

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('Folium installed')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed


In [2]:
#Placeholder for Foursquare credentials

In [3]:
address = 'Altoona,PA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.518681 -78.394736


In [4]:
search_query = 'Gym'
radius = 15759
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

In [5]:
#code to create a cleaned up df of gym venues
results = requests.get(url).json()
venues = results['response']['venues']
dataframe = json_normalize(venues)
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,St Marys Gym,Basketball Court,NaN,US,Altoona,United States,NaN,1086,"[Altoona, PA, United States]","[{'label': 'display', 'lat': 40.50918114323374...",40.509181,-78.397668,NaN,PA,524850bb498e9a683978460e
1,AAHS Fieldhouse,Sports Club,1415 6th Ave,US,Altoona,United States,btwn 13th & 14th St.,900,"[1415 6th Ave (btwn 13th & 14th St.), Altoona,...","[{'label': 'display', 'lat': 40.51077335430656...",40.510773,-78.396994,16602,PA,4b8d9d28f964a5204a0433e3
2,Gym At Homewood Suites,Gym,Poydras,US,Hollidaysburg,United States,Barrone,9626,"[Poydras (Barrone), Hollidaysburg, PA 16648, U...","[{'label': 'display', 'lat': 40.435085, 'lng':...",40.435085,-78.365639,16648,PA,4f20b8cae4b0467cd70d9403
3,Not-So-Average Joe's Gym,Gym,Woodview Drive,US,Hollidaysburg,United States,NaN,7899,"[Woodview Drive, Hollidaysburg, PA 16648, Unit...","[{'label': 'display', 'lat': 40.484025, 'lng':...",40.484025,-78.313299,16648,PA,5017d10ee4b015cce931e8ae
4,Gymboree,Kids Store,Logan Valley Mall,US,Altoona,United States,2nd Floor,5682,"[Logan Valley Mall (2nd Floor), Altoona, PA 16...","[{'label': 'display', 'lat': 40.46924120905203...",40.469241,-78.411452,16602,PA,4c2b834ef7acef3bde4fed0c
5,Gemini Gymnastics,Gymnastics Gym,1885 E Pleasant Valley Blvd,US,Altoona,United States,NaN,5636,"[1885 E Pleasant Valley Blvd, Altoona, PA 1660...","[{'label': 'display', 'lat': 40.55230352254357...",40.552304,-78.344922,16602,PA,4bd6014c6798ef3b6265648d
6,Uzelac Gymnastics,Gymnastics Gym,3519 Rte. 764,US,Duncansville,United States,NaN,6617,"[3519 Rte. 764, Duncansville, PA 16635, United...","[{'label': 'display', 'lat': 40.46469481538818...",40.464695,-78.427470,16635,PA,4e49953cd164a7c8b69c2de0


In [6]:
#the query finds some venues with the str gym in the name or address, so we'll maintain a working list of 'categories' to include in the results

myList=['Gym','Gymnastics Gym','Sports Club']

gym_list=pd.DataFrame({'categories':myList}).merge(dataframe_filtered)
gym_list

,categories,name,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Gym,Gym At Homewood Suites,Poydras,US,Hollidaysburg,United States,Barrone,9626,"[Poydras (Barrone), Hollidaysburg, PA 16648, U...","[{'label': 'display', 'lat': 40.435085, 'lng':...",40.435085,-78.365639,16648,PA,4f20b8cae4b0467cd70d9403
1,Gym,Not-So-Average Joe's Gym,Woodview Drive,US,Hollidaysburg,United States,NaN,7899,"[Woodview Drive, Hollidaysburg, PA 16648, Unit...","[{'label': 'display', 'lat': 40.484025, 'lng':...",40.484025,-78.313299,16648,PA,5017d10ee4b015cce931e8ae
2,Gymnastics Gym,Gemini Gymnastics,1885 E Pleasant Valley Blvd,US,Altoona,United States,NaN,5636,"[1885 E Pleasant Valley Blvd, Altoona, PA 1660...","[{'label': 'display', 'lat': 40.55230352254357...",40.552304,-78.344922,16602,PA,4bd6014c6798ef3b6265648d
3,Gymnastics Gym,Uzelac Gymnastics,3519 Rte. 764,US,Duncansville,United States,NaN,6617,"[3519 Rte. 764, Duncansville, PA 16635, United...","[{'label': 'display', 'lat': 40.46469481538818...",40.464695,-78.427470,16635,PA,4e49953cd164a7c8b69c2de0
4,Sports Club,AAHS Fieldhouse,1415 6th Ave,US,Altoona,United States,btwn 13th & 14th St.,900,"[1415 6th Ave (btwn 13th & 14th St.), Altoona,...","[{'label': 'display', 'lat': 40.51077335430656...",40.510773,-78.396994,16602,PA,4b8d9d28f964a5204a0433e3


In [7]:
#display gym venues on a map of Altoona
gym_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the co-ordinates for Altoona, PA

# add a red circle marker to represent the centre of Altoona
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Altoona',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(gym_map)

# add the bars as blue circle markers
for lat, lng, label in zip(gym_list.lat, gym_list.lng, gym_list.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(gym_map)

# display map
gym_map
